In [2]:
template = """BASE: ['']
TOOLBOX_MODE: "loso_cv"      # "train_and_test"  or "only_test"
TRAIN:
  BATCH_SIZE: 4
  EPOCHS: 15
  LR: 9e-4
  MODEL_FILE_NAME: AriaPPG_AriaPPG_physnet
  PLOT_LOSSES_AND_LR: True
  DATA:
    LOSO_SUBJECTS: ['P001', 'P002', 'P003', 'P004', 'P005', 'P006', 'P007', 'P008', 'P009', 'P010', 'P011', 'P012', 'P014', 'P015', 'P016', 'P017', 'P018', 'P019', 'P020', 'P021', 'P023','P024']    
    LOSO_SKIP_FOLDS: []
    FS: 30
    DATASET: AriaPPG
    DO_PREPROCESS: False               # if first time, should be true
    DATA_FORMAT: NCDHW
    DATA_PATH: "/cluster/scratch/boehis/preprocessed/median/len128/raw"                     # DiffNormalized dataset path, need to be updated
    CACHED_PATH: "/cluster/scratch/boehis/preprocessed/median/len128/prep"
    EXP_DATA_NAME: ""
    BEGIN: 0.0
    END: 1.0
    PREPROCESS:
      DATA_TYPE: ['DiffNormalized']
      LABEL_TYPE: DiffNormalized
      DO_CHUNK: True
      CHUNK_LENGTH: 128
      CROP_FACE:
        DO_CROP_FACE: False
      RESIZE:
        H: 128
        W: 128
TEST:
  METRICS: ['MAE']
  USE_LAST_EPOCH: False                      # to use provided validation dataset to find the best epoch, should be false
  DATA:
    FS: 30
    DATASET: AriaPPG
    DO_PREPROCESS: False               # if first time, should be true
    DATA_FORMAT: NCDHW
    DATA_PATH: "/cluster/scratch/boehis/preprocessed/median/len128/raw"                     # Raw dataset path, need to be updated
    CACHED_PATH: "/cluster/scratch/boehis/preprocessed/median/len128/prep"
    EXP_DATA_NAME: ""
    BEGIN: 0.0
    END: 1.0
    PREPROCESS:
      DATA_TYPE: ['DiffNormalized']         #if use physnet, should be DiffNormalized
      LABEL_TYPE: DiffNormalized
      DO_CHUNK: True
      CHUNK_LENGTH: 128
      CROP_FACE:
        DO_CROP_FACE: False
      RESIZE:
        H: 128
        W: 128
DEVICE: cuda:0
NUM_OF_GPU_TRAIN: 1
LOG:
  PATH: /cluster/scratch/boehis/runs/train_configs/PhysNet/median/multimodal/{path}
MODEL:
  DROP_RATE: 0.2
  NAME: AriaNet
  ARIANET:
    FRAME_NUM: 128
    LOSS_TRAIN: 'soft_macc'
    LOSS_VAL: 'soft_macc'
    INTEGRATION_STRATEGY: '{integration_strategy}'
    INTEGRATION_LAYERS: {integration_layer}
    SENSOR_TYPE: '{sensor_type}'
INFERENCE:
  BATCH_SIZE: 4
  EVALUATION_METHOD: ["peak detection", "FFT"]        # "FFT" or "peak detection"
  EVALUATION_WINDOW:
    USE_SMALLER_WINDOW: False        # Change this if you'd like an evaluation window smaller than the test video length
    WINDOW_SIZE: 10        # In seconds
  MODEL_PATH: ""
"""

In [4]:
configs = [
    ('add',                 [2],               'imu'),
    ('add',                 [4],               'imu'),
    ('add',                 [8],               'imu'),
    ('add',                 [10],              'imu'),
    ('cat',                 [2],               'imu'),
    ('cat',                 [4],               'imu'),
    ('cat',                 [8],               'imu'),
    ('cat',                 [10],              'imu'),
    ('adaptive_norm',       [2],               'imu'),
    ('adaptive_norm',       [4],               'imu'),
    ('adaptive_norm',       [8],               'imu'),
    ('adaptive_norm',       [10],              'imu'),
    ('cross_attention',     [2],               'imu'),
    ('cross_attention',     [4],               'imu'),
    ('cross_attention',     [8],               'imu'),
    ('cross_attention',     [10],              'imu'),
]
for integration_strategy, integration_layer, sensor_type in configs:
    integration_layer_str = "-".join(map(str, integration_layer))
    path=f"{integration_strategy}_{integration_layer_str}_{sensor_type}"

    conf_file = template.format(
        path=path,
        integration_strategy=integration_strategy,
        integration_layer=integration_layer,
        sensor_type=sensor_type,
    )
    with open(f"{path}.yaml", "w") as f:
        f.write(conf_file)
        f.close()

    print(f'sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/{path}.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/{path}.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=5000')
    

sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_2_imu.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_2_imu.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=5000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_4_imu.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_4_imu.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=5000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_8_imu.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_8_imu.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=5000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_10_imu.out --open-mode=truncate --wrap="python ./main.py --conf

In [12]:
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_10_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_10_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_2_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_2_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_4_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_4_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_8_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/adaptive_norm_8_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_10_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_10_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_4_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_4_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_10_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_10_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_2_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_2_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_4_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_4_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_8_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cat_8_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_4_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_4_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_8_quaternion_2.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_8_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000




sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_2_quaternion.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_2_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=300
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_8_quaternion.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/add_8_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=300
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_2_quaternion.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_2_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=300
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_10_quaternion.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_10_quaternion.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=300


''

In [ ]:
sbatch --output=./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_2_4_8_10_imu.out --open-mode=truncate --wrap="python ./main.py --config ./configs/train_configs/aria_ppg/PhysNet/median/multimodal/cross_attention_2_4_8_10_imu.yaml" --nodes=1 --cpus-per-task=2 --mem-per-cpu=10G --gpus=1 --time=1000